# Chapter 7: Concurrency and Parallelism (Part 2)

## Items 59-64

This notebook covers the advanced concurrency and parallelism topics from Effective Python, focusing on practical implementations of ThreadPoolExecutor, coroutines with asyncio, and true parallelism with multiprocessing.

---

## Item 59: Consider ThreadPoolExecutor When Threads Are Necessary for Concurrency

**Key Concept**: The `ThreadPoolExecutor` class combines the best aspects of threads and queues, providing simple I/O parallelism with minimal refactoring.

### Why ThreadPoolExecutor?

- **Pre-allocated threads**: No startup cost on each execution
- **Controlled resource usage**: `max_workers` parameter prevents memory blow-up
- **Automatic exception propagation**: Exceptions are properly forwarded to caller
- **Simple interface**: Easy fan-out and fan-in patterns

In [ ]:
# Basic setup for Game of Life example
ALIVE = '*'
EMPTY = '-'

class LockingGrid:
    """Thread-safe grid for Game of Life"""
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.rows = [[EMPTY] * width for _ in range(height)]
        from threading import Lock
        self.lock = Lock()
    
    def get(self, y, x):
        with self.lock:
            return self.rows[y % self.height][x % self.width]
    
    def set(self, y, x, state):
        with self.lock:
            self.rows[y % self.height][x % self.width] = state
    
    def __str__(self):
        output = ''
        for row in self.rows:
            output += ''.join(row) + '\n'
        return output.rstrip()

def count_neighbors(y, x, get):
    """Count alive neighbors for a cell"""
    n_ = get(y - 1, x + 0)  # North
    ne = get(y - 1, x + 1)  # Northeast
    e_ = get(y + 0, x + 1)  # East
    se = get(y + 1, x + 1)  # Southeast
    s_ = get(y + 1, x + 0)  # South
    sw = get(y + 1, x - 1)  # Southwest
    w_ = get(y + 0, x - 1)  # West
    nw = get(y - 1, x - 1)  # Northwest
    
    neighbor_states = [n_, ne, e_, se, s_, sw, w_, nw]
    count = sum(1 for state in neighbor_states if state == ALIVE)
    return count

def game_logic(state, neighbors):
    """Apply Game of Life rules"""
    if state == ALIVE:
        if neighbors < 2:
            return EMPTY  # Die: Too few
        elif neighbors > 3:
            return EMPTY  # Die: Too many
    else:
        if neighbors == 3:
            return ALIVE  # Regenerate
    return state

def step_cell(y, x, get, set):
    """Update a single cell"""
    state = get(y, x)
    neighbors = count_neighbors(y, x, get)
    next_state = game_logic(state, neighbors)
    set(y, x, next_state)

print("Setup complete!")

### Using ThreadPoolExecutor

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def simulate_pool(pool, grid):
    """Simulate one generation using thread pool"""
    next_grid = LockingGrid(grid.height, grid.width)
    
    futures = []
    # Fan out: Submit all cells to thread pool
    for y in range(grid.height):
        for x in range(grid.width):
            args = (y, x, grid.get, next_grid.set)
            future = pool.submit(step_cell, *args)
            futures.append(future)
    
    # Fan in: Wait for all results
    for future in futures:
        future.result()
    
    return next_grid

# Example usage
grid = LockingGrid(5, 9)
grid.set(0, 3, ALIVE)
grid.set(1, 4, ALIVE)
grid.set(2, 2, ALIVE)
grid.set(2, 3, ALIVE)
grid.set(2, 4, ALIVE)

print("Generation 0:")
print(grid)
print()

# Run 5 generations with thread pool
with ThreadPoolExecutor(max_workers=10) as pool:
    for i in range(1, 4):
        grid = simulate_pool(pool, grid)
        print(f"Generation {i}:")
        print(grid)
        print()

### Exception Propagation

ThreadPoolExecutor automatically propagates exceptions back to the caller through the `Future.result()` method:

In [ ]:
def game_logic_error(state, neighbors):
    """Version that raises an exception"""
    raise OSError('Problem with I/O')

# This will properly propagate the exception
try:
    with ThreadPoolExecutor(max_workers=10) as pool:
        task = pool.submit(game_logic_error, ALIVE, 3)
        task.result()
except OSError as e:
    print(f"Caught exception: {e}")

### Limitations of ThreadPoolExecutor

| Aspect | ThreadPoolExecutor | Alternative |
|--------|-------------------|-------------|
| **I/O Parallelism** | Limited by `max_workers` | Coroutines (asyncio) |
| **Scalability** | Poor for 10,000+ concurrent operations | Coroutines |
| **Use Case** | File I/O, no async alternative | Network I/O with asyncio |
| **Thread Startup** | Pre-allocated (efficient) | - |
| **Memory** | Controlled by `max_workers` | - |

---

## Item 60: Achieve Highly Concurrent I/O with Coroutines

**Key Concept**: Coroutines enable tens of thousands of seemingly simultaneous functions with minimal memory overhead and no complex locking.

### Why Coroutines?

**Comparison of Concurrency Approaches**:

| Feature | Threads | Coroutines |
|---------|---------|------------|
| **Startup Cost** | ~1 MB per thread | Function call |
| **Memory Usage** | ~1 MB each | <1 KB each |
| **Context Switching** | OS-managed, expensive | Event loop, cheap |
| **Locking Required** | Yes (Lock, RLock) | No (single-threaded) |
| **Max Concurrent** | Hundreds | Tens of thousands |
| **GIL Impact** | Blocks parallelism | N/A (single thread) |

### Basic Coroutine Syntax

In [ ]:
import asyncio

# Define coroutine with 'async def'
async def game_logic(state, neighbors):
    """Async version of game logic"""
    # Simulating I/O operation
    await asyncio.sleep(0.001)  # Non-blocking sleep
    
    if state == ALIVE:
        if neighbors < 2:
            return EMPTY
        elif neighbors > 3:
            return EMPTY
    else:
        if neighbors == 3:
            return ALIVE
    return state

async def step_cell_async(y, x, get, set):
    """Async version of step_cell"""
    state = get(y, x)
    neighbors = count_neighbors(y, x, get)
    next_state = await game_logic(state, neighbors)  # Use 'await' for I/O
    set(y, x, next_state)

print("Coroutine functions defined!")

### Complete Async Implementation

In [ ]:
class Grid:
    """Simple grid (no locking needed for coroutines)"""
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.rows = [[EMPTY] * width for _ in range(height)]
    
    def get(self, y, x):
        return self.rows[y % self.height][x % self.width]
    
    def set(self, y, x, state):
        self.rows[y % self.height][x % self.width] = state
    
    def __str__(self):
        return '\n'.join(''.join(row) for row in self.rows)

async def simulate(grid):
    """Simulate one generation with coroutines"""
    next_grid = Grid(grid.height, grid.width)
    
    tasks = []
    # Fan out: Create coroutines (doesn't run them yet)
    for y in range(grid.height):
        for x in range(grid.width):
            task = step_cell_async(y, x, grid.get, next_grid.set)
            tasks.append(task)
    
    # Fan in: Run all coroutines concurrently
    await asyncio.gather(*tasks)
    
    return next_grid

# Run the async simulation
async def run_simulation():
    grid = Grid(5, 9)
    grid.set(0, 3, ALIVE)
    grid.set(1, 4, ALIVE)
    grid.set(2, 2, ALIVE)
    grid.set(2, 3, ALIVE)
    grid.set(2, 4, ALIVE)
    
    print("Generation 0:")
    print(grid)
    print()
    
    for i in range(1, 4):
        grid = await simulate(grid)
        print(f"Generation {i}:")
        print(grid)
        print()

# Run in Jupyter
await run_simulation()

### Key Coroutine Concepts

**Fan-out and Fan-in Pattern**:

```
Create coroutines ──┐
                    │
                    ├─> task1 ──┐
                    ├─> task2 ──┤
                    ├─> task3 ──┼─> await asyncio.gather()
                    ├─> task4 ──┤
                    └─> task5 ──┘
```

**How Coroutines Work**:

1. **Calling a coroutine** returns a coroutine object (doesn't execute)
2. **`await` expression** pauses current coroutine, schedules awaited coroutine
3. **Event loop** rapidly switches between coroutines at `await` points
4. **`asyncio.gather()`** runs multiple coroutines concurrently and collects results

### Exception Handling in Coroutines

In [ ]:
async def game_logic_with_error(state, neighbors):
    """Coroutine that raises an exception"""
    raise OSError('Problem with I/O')

# Exceptions propagate naturally through await
try:
    await game_logic_with_error(ALIVE, 3)
except OSError as e:
    print(f"Caught exception: {e}")

### Adding More I/O Points

Easy to extend with additional I/O operations:

In [ ]:
async def count_neighbors_async(y, x, get):
    """Async version with I/O"""
    # Simulate fetching neighbor data from network
    await asyncio.sleep(0.001)
    
    neighbor_states = [
        get(y - 1, x + 0), get(y - 1, x + 1),
        get(y + 0, x + 1), get(y + 1, x + 1),
        get(y + 1, x + 0), get(y + 1, x - 1),
        get(y + 0, x - 1), get(y - 1, x - 1)
    ]
    return sum(1 for state in neighbor_states if state == ALIVE)

async def step_cell_full_async(y, x, get, set):
    """Fully async step with multiple I/O points"""
    state = get(y, x)
    neighbors = await count_neighbors_async(y, x, get)  # I/O point 1
    next_state = await game_logic(state, neighbors)      # I/O point 2
    set(y, x, next_state)

print("Extended async functions defined!")

### Benefits of Coroutines

**Decoupling**: Coroutines separate:
- **What to do** (your code logic)
- **How to do it concurrently** (event loop implementation)

**No Locking Required**: All execution in single thread, so no race conditions

**Scalability**: Can handle 10,000+ concurrent operations easily

---

## Item 61: Know How to Port Threaded I/O to asyncio

**Key Concept**: Python's async features integrate well with the language, making it straightforward to port threaded code to coroutines incrementally.

### TCP Server Example: Before (Threads)

In [ ]:
import random

# Constants
WARMER = 'Warmer'
COLDER = 'Colder'
UNSURE = 'Unsure'
CORRECT = 'Correct'

class EOFError(Exception):
    pass

class UnknownCommandError(Exception):
    pass

# Original threaded version (simplified for notebook)
class Session:
    """Guessing game session"""
    def __init__(self):
        self._clear_state(None, None)
    
    def _clear_state(self, lower, upper):
        self.lower = lower
        self.upper = upper
        self.secret = None
        self.guesses = []
    
    def set_params(self, lower, upper):
        """Set guessing range"""
        self._clear_state(lower, upper)
    
    def next_guess(self):
        """Generate next guess"""
        if self.secret is not None:
            return self.secret
        
        while True:
            guess = random.randint(self.lower, self.upper)
            if guess not in self.guesses:
                return guess
    
    def send_number(self):
        """Send guess to client"""
        guess = self.next_guess()
        self.guesses.append(guess)
        return guess
    
    def receive_report(self, decision):
        """Process client feedback"""
        last = self.guesses[-1]
        if decision == CORRECT:
            self.secret = last
        print(f'Server: {last} is {decision}')

print("Threaded session class defined!")

### TCP Server Example: After (Async)

In [ ]:
class AsyncSession:
    """Async version of guessing game session"""
    def __init__(self, reader, writer):
        self.reader = reader
        self.writer = writer
        self._clear_state(None, None)
    
    def _clear_state(self, lower, upper):
        self.lower = lower
        self.upper = upper
        self.secret = None
        self.guesses = []
    
    # Send/receive are now coroutines
    async def send(self, command):
        line = command + '\n'
        data = line.encode()
        self.writer.write(data)
        await self.writer.drain()  # Async write
    
    async def receive(self):
        line = await self.reader.readline()  # Async read
        if not line:
            raise EOFError('Connection closed')
        return line[:-1].decode()
    
    async def loop(self):
        """Main command processing loop"""
        while command := await self.receive():
            parts = command.split(' ')
            if parts[0] == 'PARAMS':
                self.set_params(int(parts[1]), int(parts[2]))
            elif parts[0] == 'NUMBER':
                await self.send_number()  # Now async
            elif parts[0] == 'REPORT':
                self.receive_report(parts[1])
            else:
                raise UnknownCommandError(command)
    
    def set_params(self, lower, upper):
        self._clear_state(lower, upper)
    
    def next_guess(self):
        if self.secret is not None:
            return self.secret
        while True:
            guess = random.randint(self.lower, self.upper)
            if guess not in self.guesses:
                return guess
    
    async def send_number(self):
        guess = self.next_guess()
        self.guesses.append(guess)
        await self.send(format(guess))  # Async send
    
    def receive_report(self, decision):
        last = self.guesses[-1]
        if decision == CORRECT:
            self.secret = last
        print(f'Server: {last} is {decision}')

print("Async session class defined!")

### Comparison: Changes Required

| Component | Threaded Version | Async Version | Change |
|-----------|------------------|---------------|--------|
| **Function Definition** | `def func():` | `async def func():` | Add `async` |
| **I/O Operations** | `file.read()` | `await reader.read()` | Add `await` |
| **Sleep** | `time.sleep(1)` | `await asyncio.sleep(1)` | Use async version |
| **Context Manager** | `@contextmanager` | `@asynccontextmanager` | Async variant |
| **For Loop** | `for x in iterable:` | `async for x in iterable:` | Add `async` |
| **Comprehension** | `[x for x in ...]` | `[x async for x in ...]` | Add `async` |
| **With Statement** | `with resource:` | `async with resource:` | Add `async` |

### Server Setup: Async Version

In [ ]:
# Async server function
async def handle_async_connection(reader, writer):
    """Handle one client connection"""
    session = AsyncSession(reader, writer)
    try:
        await session.loop()
    except EOFError:
        pass

async def run_async_server(address):
    """Run async server"""
    server = await asyncio.start_server(
        handle_async_connection, *address)
    async with server:
        await server.serve_forever()

print("Async server functions defined!")

### Client: Async Version with Language Features

In [ ]:
import contextlib
import math

class AsyncClient:
    """Async client for guessing game"""
    def __init__(self, reader, writer):
        self.reader = reader
        self.writer = writer
        self._clear_state()
    
    def _clear_state(self):
        self.secret = None
        self.last_distance = None
    
    async def send(self, command):
        line = command + '\n'
        data = line.encode()
        self.writer.write(data)
        await self.writer.drain()
    
    async def receive(self):
        line = await self.reader.readline()
        if not line:
            raise EOFError('Connection closed')
        return line[:-1].decode()
    
    # Async context manager
    @contextlib.asynccontextmanager
    async def session(self, lower, upper, secret):
        print(f'Guess a number between {lower} and {upper}! '
              f"Shhhhh, it's {secret}.")
        self.secret = secret
        await self.send(f'PARAMS {lower} {upper}')
        try:
            yield
        finally:
            self._clear_state()
            await self.send('PARAMS 0 -1')
    
    # Async generator
    async def request_numbers(self, count):
        for _ in range(count):
            await self.send('NUMBER')
            data = await self.receive()
            yield int(data)
            if self.last_distance == 0:
                return
    
    async def report_outcome(self, number):
        new_distance = math.fabs(number - self.secret)
        decision = UNSURE
        
        if new_distance == 0:
            decision = CORRECT
        elif self.last_distance is None:
            pass
        elif new_distance < self.last_distance:
            decision = WARMER
        elif new_distance > self.last_distance:
            decision = COLDER
        
        self.last_distance = new_distance
        await self.send(f'REPORT {decision}')
        return decision

print("Async client class defined!")

### Running Client: Async Features

In [ ]:
async def run_async_client(address):
    """Run async client with multiple async language features"""
    # Open connection
    streams = await asyncio.open_connection(*address)
    client = AsyncClient(*streams)
    
    # Async context manager + async list comprehension
    async with client.session(1, 5, 3):
        results = [(x, await client.report_outcome(x))
                   async for x in client.request_numbers(5)]
    
    # Async context manager + async for loop
    async with client.session(10, 15, 12):
        async for number in client.request_numbers(5):
            outcome = await client.report_outcome(number)
            results.append((number, outcome))
    
    # Clean up
    _, writer = streams
    writer.close()
    await writer.wait_closed()
    
    return results

print("Async client runner defined!")

### Putting It All Together

In [ ]:
async def main_async():
    """Main async program"""
    address = ('127.0.0.1', 4321)
    
    # Start server in background
    server = run_async_server(address)
    asyncio.create_task(server)
    
    # Give server time to start
    await asyncio.sleep(0.1)
    
    # Run client
    results = await run_async_client(address)
    
    # Print results
    for number, outcome in results:
        print(f'Client: {number} is {outcome}')

# Note: Would run with asyncio.run(main_async()) in regular Python
print("Main async function defined!")

### Async Language Features Summary

Python provides async versions of many language features:

```python
# Async context manager
async with resource:
    ...

# Async for loop  
async for item in async_iterator:
    ...

# Async list comprehension
results = [x async for x in async_iterator]

# Async generator
async def generator():
    yield value

# Async context manager decorator
@contextlib.asynccontextmanager
async def manager():
    yield
```

---

## Item 62: Mix Threads and Coroutines to Ease the Transition to asyncio

**Key Concept**: Use `run_in_executor` and `run_coroutine_threadsafe` to incrementally migrate from threads to coroutines.

### Two Migration Approaches

**Top-Down Migration**:
```
Main Entry Point (convert to async)
        ↓
High-level functions (convert to async)
        ↓
Mid-level functions (convert to async)
        ↓
Low-level functions (keep blocking I/O)
```

**Bottom-Up Migration**:
```
Low-level functions (convert to async)
        ↑
Mid-level functions (convert to async)
        ↑
High-level functions (convert to async)
        ↑
Main Entry Point (convert to async)
```

### Example: Log File Tailer

In [ ]:
import time

class NoNewData(Exception):
    pass

def readline(handle):
    """Read next line if available"""
    offset = handle.tell()
    handle.seek(0, 2)  # Seek to end
    length = handle.tell()
    
    if length == offset:
        raise NoNewData
    
    handle.seek(offset, 0)
    return handle.readline()

def tail_file(handle, interval, write_func):
    """Tail file in thread"""
    while not handle.closed:
        try:
            line = readline(handle)
        except NoNewData:
            time.sleep(interval)
        else:
            write_func(line)

print("Blocking I/O functions defined!")

### Top-Down Approach: run_in_executor

In [ ]:
async def run_tasks_mixed(handles, interval, output_path):
    """Mixed threads and coroutines approach"""
    loop = asyncio.get_event_loop()
    
    with open(output_path, 'wb') as output:
        async def write_async(data):
            output.write(data)
        
        def write(data):
            # Run coroutine from thread
            coro = write_async(data)
            future = asyncio.run_coroutine_threadsafe(coro, loop)
            future.result()
        
        tasks = []
        for handle in handles:
            # Run blocking function in thread pool
            task = loop.run_in_executor(
                None, tail_file, handle, interval, write)
            tasks.append(task)
        
        await asyncio.gather(*tasks)

print("Mixed approach defined!")

### Bottom-Up Approach: Wrap Blocking Code

In [ ]:
async def tail_async(handle, interval, write_func):
    """Async version using run_in_executor for blocking parts"""
    loop = asyncio.get_event_loop()
    
    while not handle.closed:
        try:
            # Run blocking readline in thread pool
            line = await loop.run_in_executor(
                None, readline, handle)
        except NoNewData:
            await asyncio.sleep(interval)
        else:
            await write_func(line)

async def run_tasks(handles, interval, output_path):
    """Fully async version"""
    with open(output_path, 'wb') as output:
        async def write_async(data):
            output.write(data)
        
        tasks = []
        for handle in handles:
            coro = tail_async(handle, interval, write_async)
            task = asyncio.create_task(coro)
            tasks.append(task)
        
        await asyncio.gather(*tasks)

print("Fully async approach defined!")

### Key Functions for Mixing Threads and Coroutines

| Function | Purpose | Direction |
|----------|---------|----------|
| **`run_in_executor()`** | Run blocking function in thread | Coroutine → Thread |
| **`run_coroutine_threadsafe()`** | Run coroutine from thread | Thread → Coroutine |
| **`run_until_complete()`** | Block thread until coroutine done | Thread → Coroutine |
| **`asyncio.create_task()`** | Schedule coroutine | Coroutine → Coroutine |

---

## Item 63: Avoid Blocking the asyncio Event Loop to Maximize Responsiveness

**Key Concept**: System calls in coroutines can block the event loop and hurt responsiveness. Move slow operations to separate threads.

### Detecting Blocking Operations

In [ ]:
import time

async def slow_coroutine():
    """BAD: Blocks event loop"""
    time.sleep(0.5)  # This blocks!

# Debug mode will detect blocking
# asyncio.run(slow_coroutine(), debug=True)
# Output: "took 0.503 seconds" warning

print("Blocking example defined (don't run it!)")

### Solution: Move Blocking I/O to Separate Thread

In [ ]:
from threading import Thread

class WriteThread(Thread):
    """Thread for handling blocking file I/O"""
    def __init__(self, output_path):
        super().__init__()
        self.output_path = output_path
        self.output = None
        self.loop = asyncio.new_event_loop()
    
    def run(self):
        """Run event loop in this thread"""
        asyncio.set_event_loop(self.loop)
        with open(self.output_path, 'wb') as self.output:
            self.loop.run_forever()
        # Final cleanup
        self.loop.run_until_complete(asyncio.sleep(0))
    
    async def real_write(self, data):
        """Actual write (runs in worker thread)"""
        self.output.write(data)
    
    async def write(self, data):
        """Thread-safe write (callable from main loop)"""
        coro = self.real_write(data)
        future = asyncio.run_coroutine_threadsafe(
            coro, self.loop)
        await asyncio.wrap_future(future)
    
    async def real_stop(self):
        """Stop event loop"""
        self.loop.stop()
    
    async def stop(self):
        """Thread-safe stop"""
        coro = self.real_stop()
        future = asyncio.run_coroutine_threadsafe(
            coro, self.loop)
        await asyncio.wrap_future(future)
    
    # Async context manager support
    async def __aenter__(self):
        loop = asyncio.get_event_loop()
        await loop.run_in_executor(None, self.start)
        return self
    
    async def __aexit__(self, *_):
        await self.stop()

print("WriteThread class defined!")

### Using WriteThread

In [ ]:
async def run_fully_async(handles, interval, output_path):
    """Fully async with blocking I/O in separate thread"""
    async with WriteThread(output_path) as output:
        tasks = []
        for handle in handles:
            # tail_async defined earlier
            coro = tail_async(handle, interval, output.write)
            task = asyncio.create_task(coro)
            tasks.append(task)
        
        await asyncio.gather(*tasks)

print("Fully async runner defined!")

### Event Loop Responsiveness Patterns

**Good Patterns** (Don't block event loop):
```python
# Async I/O
data = await reader.read(1024)

# Async sleep
await asyncio.sleep(1)

# Run blocking code in thread pool
await loop.run_in_executor(None, blocking_func)
```

**Bad Patterns** (Block event loop):
```python
# Blocking I/O
data = file.read()  # BAD!

# Blocking sleep  
time.sleep(1)  # BAD!

# CPU-intensive work
result = expensive_computation()  # BAD!
```

---

## Item 64: Consider concurrent.futures for True Parallelism

**Key Concept**: Use `ProcessPoolExecutor` to achieve true CPU parallelism by running code in separate processes.

### The Parallelism Problem

**Threading Limitations**:
- GIL prevents true parallelism
- Only one thread executes Python at a time
- Good for I/O, bad for CPU-bound work

In [ ]:
# Example: CPU-intensive function
def gcd(pair):
    """Find greatest common divisor"""
    a, b = pair
    low = min(a, b)
    for i in range(low, 0, -1):
        if a % i == 0 and b % i == 0:
            return i
    assert False, 'Not reachable'

NUMBERS = [
    (1963309, 2265973), (2030677, 3814172),
    (1551645, 2229620), (2039045, 2020802),
    (1823712, 1924928), (2293129, 1020491),
]

print("GCD function and test data defined!")

### Serial Execution (Baseline)

In [ ]:
import time

start = time.time()
results = list(map(gcd, NUMBERS))
end = time.time()

print(f'Serial execution took {end - start:.3f} seconds')
print(f'Results: {results}')

### Thread-Based Parallelism (No Speedup)

In [ ]:
from concurrent.futures import ThreadPoolExecutor

start = time.time()
pool = ThreadPoolExecutor(max_workers=2)
results = list(pool.map(gcd, NUMBERS))
end = time.time()

print(f'Thread execution took {end - start:.3f} seconds')
print('Note: Same or slower due to GIL!')

### Process-Based Parallelism (Real Speedup!)

In [ ]:
from concurrent.futures import ProcessPoolExecutor

# Note: In notebook, this might not work due to pickling limitations
# In a regular .py file, this would show real speedup

if __name__ == '__main__':  # Required for multiprocessing
    start = time.time()
    pool = ProcessPoolExecutor(max_workers=2)
    results = list(pool.map(gcd, NUMBERS))
    end = time.time()
    
    print(f'Process execution took {end - start:.3f} seconds')
    print('Expected: ~50% of serial time on dual-core machine!')

### How ProcessPoolExecutor Works

**Step-by-Step Process**:

```
1. Input data ────────> pickle.dumps()
                             │
2. Binary data ──────> socket send()
                             │
3. Child process ────> pickle.loads()
                             │
4. Python objects ───> import module
                             │
5. Run function ─────> gcd(data)
                             │
6. Result ───────────> pickle.dumps()
                             │
7. Binary data ──────> socket send()
                             │
8. Parent process ───> pickle.loads()
                             │
9. Python objects ───> merge results
```

### Performance Comparison

| Approach | Execution Time | Speedup | Use Case |
|----------|---------------|---------|----------|
| **Serial** | 1.0x (baseline) | 1.0x | Simple programs |
| **Threads** | 1.0-1.2x | None | I/O-bound only |
| **Processes** | 0.5-0.6x | ~2x | CPU-bound work |
| **C Extension** | 0.1-0.3x | ~5-10x | Critical hotspots |

### When to Use ProcessPoolExecutor

**Good Use Cases** (Isolated, High-Leverage):
- Mathematical computations
- Image/video processing
- Data compression
- Cryptographic operations
- Scientific simulations

**Requirements**:
1. **Isolated**: Functions don't share state
2. **High-leverage**: Small data in, lots of computation, small data out
3. **Picklable**: All inputs and outputs can be serialized

**Poor Use Cases**:
- Functions that share lots of state
- Large data transfer overhead
- Non-picklable objects (sockets, file handles)
- Frequent inter-process communication

### Practical Example: Image Processing

In [ ]:
def process_image(data):
    """
    Example: Process image data
    - Input: Small (image ID or path)
    - Computation: Large (image processing)
    - Output: Small (processed result)
    """
    image_path = data
    # Load image, apply filters, compute features
    # ... expensive computation ...
    return {"processed": True, "features": [...]}

# Good use case: Process many images in parallel
image_paths = [f"image_{i}.jpg" for i in range(100)]

# with ProcessPoolExecutor(max_workers=4) as pool:
#     results = list(pool.map(process_image, image_paths))

print("Image processing example defined!")

### Alternative: C Extensions

When to consider C extensions:
- ProcessPoolExecutor overhead is too high
- Need maximum performance
- Algorithm is well-defined and stable

**Trade-offs**:
```python
# Python with ProcessPoolExecutor
pros = ["Easy to write", "Easy to debug", "Portable"]
cons = ["Pickling overhead", "2-4x slower than C"]

# C Extension
pros = ["Maximum speed", "Can bypass GIL", "Access to C libraries"]
cons = ["Complex", "Hard to debug", "Requires testing", "Platform-specific"]
```

---

## Summary: Concurrency and Parallelism Part 2

### Decision Tree

```
Need concurrency?
    │
    ├─ I/O-bound?
    │   ├─ < 100 concurrent: ThreadPoolExecutor
    │   └─ > 100 concurrent: asyncio coroutines
    │
    └─ CPU-bound?
        ├─ Isolated functions: ProcessPoolExecutor
        └─ Critical hotspot: C extension
```

### Key Takeaways

1. **ThreadPoolExecutor**: Best for limited I/O parallelism with minimal refactoring

2. **Coroutines (asyncio)**: Best for high I/O concurrency (1000s of operations)

3. **Porting to asyncio**: Use `run_in_executor` and `run_coroutine_threadsafe` for incremental migration

4. **Event Loop**: Avoid blocking with slow system calls; use separate threads for blocking I/O

5. **ProcessPoolExecutor**: Use for CPU-bound, isolated, high-leverage tasks

6. **C Extensions**: Last resort for maximum performance after exhausting Python options

### Things to Remember

**Item 59 - ThreadPoolExecutor**:
- Pre-allocates threads to avoid startup cost
- `max_workers` prevents memory blow-up
- Automatically propagates exceptions
- Limited scalability compared to coroutines

**Item 60 - Coroutines**:
- Use `async def` for coroutines
- Use `await` to pause and resume
- No locking needed (single-threaded)
- Can handle 10,000+ concurrent operations
- `asyncio.gather()` for fan-in pattern

**Item 61 - Porting to asyncio**:
- Python provides async versions of most language features
- Top-down: Start at entry points, use `run_in_executor`
- Bottom-up: Start at leaves, use `run_until_complete`
- Can mix threads and coroutines during migration

**Item 62 - Mixing Threads and Coroutines**:
- `run_in_executor`: Run blocking code from coroutine
- `run_coroutine_threadsafe`: Run coroutine from thread
- Use for incremental migration
- Eventually eliminate mixing for cleaner code

**Item 63 - Event Loop Responsiveness**:
- System calls block event loop
- Use `debug=True` to detect blocking
- Move blocking I/O to separate threads
- Use `asyncio.wrap_future` to await thread results

**Item 64 - True Parallelism**:
- GIL prevents thread parallelism for CPU-bound work
- `ProcessPoolExecutor` runs code in separate processes
- Each process has own GIL, enabling parallelism
- High overhead from pickling and IPC
- Best for isolated, high-leverage tasks
- Consider C extensions for critical hotspots